In [189]:
import matplotlib.pyplot as plt
%matplotlib inline
import librosa.display
import pandas as pd
import numpy as np

import librosa
from librosa.util import fix_length

import torch
torch.set_num_threads(1)

from IPython.display import Audio
from pprint import pprint



In [190]:
sr = 16000
audio_file_path='audio_wav_fold/fold112/112-al-ikhlas_4.wav'
librosa_audio_data,librosa_sample_rate=librosa.load(audio_file_path, sr= 16000)

librosa_audio_data.shape

(253702,)

VOICE ACTIVITY DETECTION USING SILERO-VAD MODEL & EXTRACT START-END POINTS


In [191]:
SAMPLING_RATE = 16000

USE_ONNX = False # change this to True if you want to test onnx model
if USE_ONNX:
    !pip install -q onnxruntime
    
 
model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True,
                              onnx=USE_ONNX)

(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils


# get speech timestamps from full audio file
speech_timestamps = get_speech_timestamps(librosa_audio_data, model, sampling_rate=SAMPLING_RATE)
pprint(speech_timestamps)

################################# Seperate START and END points from Speech_timpstamps ##########################################

# Using list comprehension
# Get values of particular key in list of dictionaries
start = [ sub['start'] for sub in speech_timestamps ]
 
# printing result
print("The values corresponding to Start : " + str(start))

# Using list comprehension
# Get values of particular key in list of dictionaries
end = [ sub['end'] for sub in speech_timestamps ]
 
# printing result
print("The values corresponding to End : " + str(end))

start = np.array(start)
end = np.array(end)

Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to C:\Users\aleem/.cache\torch\hub\master.zip


[{'end': 44000, 'start': 1568},
 {'end': 92128, 'start': 51232},
 {'end': 132576, 'start': 99360},
 {'end': 180192, 'start': 137760},
 {'end': 246240, 'start': 184864}]
The values corresponding to Start : [1568, 51232, 99360, 137760, 184864]
The values corresponding to End : [44000, 92128, 132576, 180192, 246240]


EXTRACTING AMPLITUDES IMPORTANT FOR US

In [192]:
def extract_amplitudes(audio_data,length,start,end):
    
    extracted_librosa_audio_data = []
    
    for i in range(length):
        extracted_librosa_audio_data.append([audio_data[start[i]:end[i]]])
    return np.array(extracted_librosa_audio_data)

In [193]:
extracted_librosa_audio_data = extract_amplitudes(librosa_audio_data,len(start),start,end)
# import pandas as pd
# df = pd.DataFrame(extracted_librosa_audio_data)
# df.fillna(0).values.tolist()
extracted_librosa_audio_data.shape

C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  import sys


(5, 1)

SAVE VAD FILES AS A SEPERATE .WAV FILES

In [194]:
import numpy as np
import scipy.io.wavfile
from scipy.io.wavfile import write

rate = 16000

filenames = []


n=len(start)

for i in range(1,len(start)+1):
    filename = "file4_{}.wav".format(i)
    filenames.append(filename)
    
print(filenames)


for count,i in enumerate(range(len(start)),1):
    savefile = extracted_librosa_audio_data[i][0]
    filename = filenames[i]
    write(filename = 'extracted_audio_wav_fold/fold112/'+ filename, rate = rate, data = savefile)

['file4_1.wav', 'file4_2.wav', 'file4_3.wav', 'file4_4.wav', 'file4_5.wav']
